In [29]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from collections import Counter


In [30]:
property_df = pd.read_csv("../data/curated/property_isochromes.csv")

In [31]:
property_df.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,https://www.domain.com.au/406a-62-66-nicholson...,406A/62 - 66 Nicholson Street Footscray VIC 3011,280.0,1,1,0,Apartment / Unit / Flat,locat matter,213031348,144.899514,-37.798485,213031348.0,916.0,355.0,2296.0,1763.0,16695.0,Footscray,10.0


In [36]:
y = property_df['cost_text']
type(y[0])
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
X = property_df.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021"], axis=1)
X.astype({'cost_text': 'float', 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "int64", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          "SA2_NAME21": "category"}).dtypes

l = [(elem, type(elem)) for elem in X.loc[0]]
print(l)
X.head(1)

[(280.0, <class 'numpy.float64'>), (1, <class 'numpy.int64'>), (1, <class 'numpy.int64'>), (0, <class 'numpy.int64'>), ('Apartment / Unit / Flat', <class 'str'>), (916.0, <class 'numpy.float64'>), (355.0, <class 'numpy.float64'>), (2296.0, <class 'numpy.float64'>), (1763.0, <class 'numpy.float64'>), (16695.0, <class 'numpy.float64'>), ('Footscray', <class 'str'>), (10.0, <class 'numpy.float64'>)]


,cost_text,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,280.0,1,1,0,Apartment / Unit / Flat,916.0,355.0,2296.0,1763.0,16695.0,Footscray,10.0


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.compose import ColumnTransformer

numeric_features = ['cost_text', 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "duration_mins", 
                  "Bed", 
                  "Bath", 
                  "Park"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

clf = MLPClassifier(max_iter=2000)
#this way we don't cheat. Read more on pipelines https://scikit-learn.org/stable/modules/compose.html
pipeline = Pipeline([('transformer', preprocessor), ('estimator', clf)])
print('Cross-val noncheating standardised features acc:', np.mean(cross_val_score(pipeline, X, y, cv=5)))



Cross-val noncheating standardised features acc: nan


/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 752, in fit
    ret

In [38]:
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 0)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
# fit the data
rf_model.fit(X_train, y_train)  

ValueError: could not convert string to float: 'Apartment / Unit / Flat'

In [ ]:
num_clusters = 3

gmm = GaussianMixture(n_components=num_clusters, covariance_type='full')
gmm.fit(data)

plt.scatter(data[:,0], data[:,1])
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.scatter(gmm.means_[:,0],gmm.means_[:,1])
plt.scatter(means[:,0],means[:,1])
plt.legend(['Data','Estimated mean','True mean'])
plt.show()

print('weights:\n {}\n'.format(gmm.weights_))
print('means:\n {}\n'.format(gmm.means_))
print('covariances:\n {}\n'.format(gmm.covariances_))